In [1]:
import pandas as pd

import pandas as pd
from src.data.ingestion import load_tickets

In [2]:
from src.data.graph_rag_ingest import build_support_graph
from src.data.graph_rag_query import top_solutions_for_issue

In [3]:
json_data = load_tickets("../data/raw/support_tickets.json")

In [4]:
Data_tickets = json_data[0]

In [7]:
#Data_tickets[:2]

In [6]:
support_graph = build_support_graph(Data_tickets[:])

print("nodes:", support_graph.number_of_nodes())
print("edges:", support_graph.number_of_edges())

print(top_solutions_for_issue(support_graph, "Account Management", "Upgrade", top_k=5))

nodes: 110078
edges: 517739
[('solution:DATA_REPAIR', 0.6649484536082474, 388), ('solution:ENVIRONMENT_ISSUE', 0.6240208877284595, 383), ('solution:DUPLICATE', 0.6222826086956522, 368), ('solution:WORKAROUND', 0.6208955223880597, 335), ('solution:CONFIG_CHANGE', 0.6186666666666667, 375)]


In [14]:
#support_graph.nodes

In [15]:
import random
import networkx as nx
import matplotlib.pyplot as plt

def visualize_subgraph(
    support_graph: nx.Graph,
    center_node: str,
    hops: int = 2,
    max_nodes: int = 80,
    seed: int = 42,
):
    if center_node not in support_graph:
        raise ValueError(f"{center_node} not found in graph")

    # collect nodes within N hops (BFS)
    nodes = {center_node}
    frontier = {center_node}
    for _ in range(hops):
        nxt = set()
        for n in frontier:
            nxt.update(support_graph.neighbors(n))
        nodes.update(nxt)
        frontier = nxt

    nodes = list(nodes)

    # cap nodes for readability
    if len(nodes) > max_nodes:
        random.seed(seed)
        nodes = [center_node] + random.sample([n for n in nodes if n != center_node], max_nodes - 1)

    sub = support_graph.subgraph(nodes).copy()

    # layout
    plt.figure(figsize=(14, 9))
    pos = nx.spring_layout(sub, seed=seed, k=0.7)

    # optional: color by "kind" (we don’t set colors explicitly; matplotlib will default)
    kinds = nx.get_node_attributes(sub, "kind")
    kind_values = sorted(set(kinds.values())) if kinds else []
    if kind_values:
        for kind in kind_values:
            kind_nodes = [n for n in sub.nodes if kinds.get(n) == kind]
            nx.draw_networkx_nodes(sub, pos, nodelist=kind_nodes, node_size=700, alpha=0.9)
    else:
        nx.draw_networkx_nodes(sub, pos, node_size=700, alpha=0.9)

    nx.draw_networkx_edges(sub, pos, alpha=0.5, width=1.5)

    # shorter labels
    def short_label(node: str) -> str:
        for prefix, tag in [("ticket:", ""), ("product:", "P:"), ("module:", "M:"), ("issue:", "I:"), ("solution:", "S:"), ("error:", "E:")]:
            if node.startswith(prefix):
                return tag + node[len(prefix):]
        return node

    labels = {n: short_label(str(n)) for n in sub.nodes}
    nx.draw_networkx_labels(sub, pos, labels=labels, font_size=9)

    # show relationship labels if present
    edge_rels = nx.get_edge_attributes(sub, "rel")
    if edge_rels:
        nx.draw_networkx_edge_labels(sub, pos, edge_labels=edge_rels, font_size=8)

    plt.title(f"Graph-RAG subgraph around: {short_label(center_node)} (≤{max_nodes} nodes, {hops}-hop)")
    plt.axis("off")
    plt.show()


In [17]:
# issue_node = "issue:Account Management|Upgrade"
# visualize_subgraph(support_graph, center_node=issue_node, hops=2, max_nodes=80)

In [18]:
### by using pyvis in HTML 

from pyvis.network import Network
import networkx as nx


def export_interactive_html(
    support_graph: nx.Graph,
    center_node: str,
    hops: int = 2,
    max_nodes: int = 200,
    out_html: str = "graph_rag.html",
):
    # --- collect subgraph ---
    nodes = {center_node}
    frontier = {center_node}
    for _ in range(hops):
        nxt = set()
        for n in frontier:
            nxt.update(support_graph.neighbors(n))
        nodes.update(nxt)
        frontier = nxt

    nodes = list(nodes)
    if len(nodes) > max_nodes:
        nodes = nodes[:max_nodes]

    sub = support_graph.subgraph(nodes).copy()

    # --- create interactive graph ---
    net = Network(height="750px", width="100%", notebook=True, directed=False)
    net.from_nx(sub)

    # --- add relationship labels to edges ---
    edge_lookup = {}
    for u, v, data in sub.edges(data=True):
        rel = data.get("rel")
        if rel:
            edge_lookup[(u, v)] = rel
            edge_lookup[(v, u)] = rel  # undirected graph

    for edge in net.edges:
        key = (edge["from"], edge["to"])
        rel = edge_lookup.get(key)
        if rel:
            edge["label"] = rel
            edge["title"] = rel

    net.show(out_html)
    return out_html


In [ ]:
# export_interactive_html(
#     support_graph,
#     center_node="issue:Account Management|Upgrade",
#     hops=2,
#     out_html="graph_rag.html"
# )
